# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Valerie  
Date: Aug 17, 2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? False


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [3]:
SYMBOL = 'PLTR'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d', auto_adjust=False).reset_index()[['Date','Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

[*********************100%***********************]  1 of 1 completed


{'missing': [], 'shape': (63, 2), 'na_total': 0}

In [4]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved ..\data\raw\api_source-yfinance_symbol-PLTR_20250817-204114.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [5]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'  # TODO: replace with permitted page
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'}
# try:
#     resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
#     soup = BeautifulSoup(resp.text, 'html.parser')
#     rows = [[c.get_text(strip=True) for c in tr.find_all(['tr', 'td'])] for tr in soup.find_all('td')]
#     header, *data = [r for r in rows if r]
#     df_scrape = pd.DataFrame(data, columns=header)
# except Exception as e:
#     print('Scrape failed, using inline demo table:', e)
#     html = '<span class="p--small" data-qa="discovery-media-list-item-title">Nobody 2</span>'
#     soup = BeautifulSoup(html, 'html.parser')
#     rows = [[c.get_text(strip=True) for c in tr.find_all(['tr', 'td'])] for tr in soup.find_all('td')]
#     header, *data = [r for r in rows if r]
#     df_scrape = pd.DataFrame(data, columns=header)

# if 'Price' in df_scrape.columns:
#     df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
# v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

In [6]:
resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
table = soup.find_all('table', class_='wikitable sortable')[1]
world_titles = table.find_all('th')
world_table_titles = [title.text.strip() for title in world_titles]
df = pd.DataFrame(columns=world_table_titles)
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


In [7]:
column_data = table.find_all('tr')
for row in column_data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    length = len(df)
    df.loc[length] = individual_row_data
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"648,125",6.0%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"574,785",11.9%,"1,525,000","Seattle, Washington"
2,3,Apple,Electronics industry,"383,482",-2.8%,"161,000","Cupertino, California"
3,4,UnitedHealth Group,Healthcare,"371,622",14.6%,"440,000","Minnetonka, Minnesota"
4,5,Berkshire Hathaway,Conglomerate,"364,482",20.7%,"396,500","Omaha, Nebraska"
...,...,...,...,...,...,...,...
95,96,TIAA,Financials,"45,735",11.8%,"16,023","New York City, New York"
96,97,CHS,Agriculture cooperative,"45,590",-4.6%,"10,609","Inver Grove Heights, Minnesota"
97,98,Bristol-Myers Squibb,Pharmaceutical industry,"45,006",-2.5%,"34,100","New York City, New York"
98,99,Dow Chemical Company,Chemical industry,"44,622",-21.6%,"35,900","Midland, Michigan"


In [8]:
_ = save_csv(df, prefix='scrape', site='wiki', table='large_com')

Saved ..\data\raw\scrape_site-wiki_table-large_com_20250817-204115.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.